## Initialization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# create directory for data
!mkdir data

# unzip files from zip folder
!unzip "/content/drive/MyDrive/PG/PB/Projekt badawczy - terminy medyczne/001_004.zip" -d "data/"

mkdir: cannot create directory ‘data’: File exists
Archive:  /content/drive/MyDrive/PG/PB/Projekt badawczy - terminy medyczne/001_004.zip
replace data/001_004/001_004_online.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Package installation

In [3]:
!pip install resemble-enhance --upgrade


## Imports

In [4]:
import pathlib
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment, effects
import os
import IPython.display as ipd

# RESEMBLE ENHANCE
import gradio as gr
import torch
import torchaudio

from resemble_enhance.enhancer.inference import denoise, enhance

[2024-10-03 14:11:37,297] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
data_directory = "./data/001_004/"
data_path = pathlib.Path(data_directory)
files = list(data_path.glob('*.wav'))

## Functionality definitions

In [11]:
class Audio:
  def __init__(self, audio, freq):
    self.audio = audio
    self.sr = freq

  def display_play_button(self):
    display(ipd.Audio(data=self.audio, rate=self.sr))

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

def _fn(path, solver, nfe, tau, denoising):
    if path is None:
        return None, None

    solver = solver.lower()
    nfe = int(nfe)
    lambd = 0.9 if denoising else 0.1

    dwav, sr = torchaudio.load(path)
    dwav = dwav.mean(dim=0)

    wav1, new_sr = denoise(dwav, sr, device)
    wav2, new_sr = enhance(dwav, sr, device, nfe=nfe, solver=solver, lambd=lambd, tau=tau)

    wav1 = wav1.cpu().numpy()
    wav2 = wav2.cpu().numpy()

    return Audio(wav2, new_sr)


### Enhancer settings

## Sample processing

### Execute enhancer

In [ ]:
original_audio = []
enhanced_audio = []

solver = "Midpoint"
nfe = 64
tau = 0.5
denoising = False

for file in files:
  enhanced_audio_sample = _fn(file, solver, nfe, tau, denoising)
  # original_audio.append(Audio(audio2[1], audio2[0]))
  enhanced_audio.append(enhanced_audio_sample)
  break

# original_audio[0].display_play_button()
# enhanced_audio[0].display_play_button()

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# original_file, sr = librosa.load(files[0], sr=None)

# enhanced_file = v1[1]
# enhanced_file_fq = v1[0]

# display(ipd.Audio(data=original_file, rate=sr))
# display(ipd.Audio(data=enhanced_file, rate=enhanced_file_fq))
# # print(v1[1])
for original_file in original_files:
  original_file.display_play_button()